In [3]:
import logging

import click
import fsspec
import geopandas as gpd
from deafrica_waterbodies.cli.logs import logging_setup
from deafrica_waterbodies.datasets import get_datasets_ids
from deafrica_waterbodies.io import check_if_s3_uri

In [4]:
import deafrica_waterbodies

In [5]:
import os

# These are the default AWS configurations for the Analysis Sandbox.
# that are set in the environmnet variables.
aws_default_config = {
    # "AWS_NO_SIGN_REQUEST": "YES",
    "AWS_SECRET_ACCESS_KEY": "fake",
    "AWS_ACCESS_KEY_ID": "fake",
}

# To access public bucket, need to remove the AWS credentials in
# the environment variables or the following error will occur.
# PermissionError: The AWS Access Key Id you provided does not exist in our records.

for key in aws_default_config.keys():
    if key in os.environ:
        del os.environ[key]

In [6]:
verbose = 1
aoi_vector_file = "data/SenegalBasin.geojson"
num_workers = 16
dataset_ids_text_file = "s3://deafrica-waterbodies-dev/test_out_dir/0-0-1/dataset_ids.txt"

In [7]:
# Set up logger.
logging_setup(verbose=verbose)
_log = logging.getLogger(__name__)

In [8]:
# Support pathlib Paths.
aoi_vector_file = str(aoi_vector_file)
dataset_ids_text_file = str(dataset_ids_text_file)

In [11]:
aoi_vector_file

'data/continental.geojson'

In [9]:
# Load the area of interest as a GeoDataFrame.
if aoi_vector_file is not None:
    try:
        aoi_gdf = gpd.read_file(aoi_vector_file)
    except Exception as error:
        _log.exception(f"Could not read the file {aoi_vector_file}")
        raise error
else:
    aoi_gdf = None

[2023-10-19 23:17:12,637] {collection.py:243} ERROR - data/continental.geojson: No such file or directory
[2023-10-19 23:17:12,642] {2506209837.py:6} ERROR - Could not read the file data/continental.geojson
Traceback (most recent call last):
  File "fiona/ogrext.pyx", line 136, in fiona.ogrext.gdal_open_vector
  File "fiona/_err.pyx", line 291, in fiona._err.exc_wrap_pointer
fiona._err.CPLE_OpenFailedError: data/continental.geojson: No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_104/2506209837.py", line 4, in <module>
    aoi_gdf = gpd.read_file(aoi_vector_file)
  File "/usr/local/lib/python3.10/dist-packages/geopandas/io/file.py", line 281, in _read_file
    return _read_file_fiona(
  File "/usr/local/lib/python3.10/dist-packages/geopandas/io/file.py", line 322, in _read_file_fiona
    with reader(path_or_bytes, **kwargs) as features:
  File "/usr/local/lib/python3.10/dist-packag

DriverError: data/continental.geojson: No such file or directory

In [10]:
# Get the WOfS All Time Summary scene ids for the scenes whose extent
# intersects with the area of interest.
dataset_ids = get_datasets_ids(aoi_gdf=aoi_gdf, num_workers=num_workers)

NameError: name 'aoi_gdf' is not defined

In [8]:
# Set the filesystem to use.
if check_if_s3_uri(dataset_ids_text_file):
    fs = fsspec.filesystem("s3")
else:
    fs = fsspec.filesystem("file")

In [9]:
# Write the dataset ids to the text file.
with fs.open(dataset_ids_text_file, "w") as file:
    for dataset_id in dataset_ids:
        file.write(f"{dataset_id}\n")

_log.info(f"Dataset IDs written to: {dataset_ids_text_file}.")

[2023-10-06 16:39:44,855] {credentials.py:620} INFO - Found credentials in shared credentials file: ~/.aws/credentials
[2023-10-06 16:39:45,022] {2876337735.py:6} INFO - Dataset IDs written to: s3://deafrica-waterbodies-dev/test_out_dir/0-0-1/dataset_ids.txt.
